In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.metrics import average_precision_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,precision_recall_curve,confusion_matrix,accuracy_score,roc_curve,f1_score,auc
%matplotlib inline

# Read Data

In [ ]:
df=pd.read_csv('/kaggle/input/heart-disease-uci/heart.csv')
df.head(5)

In [ ]:
df.describe()

There is no missing values in dataset

In [ ]:
df.info()

In [ ]:
plt.figure(figsize=(14,10))
sns.heatmap(df.corr(),annot=True,fmt='.3f',linewidths=2)
plt.show()

In [ ]:
df.hist(figsize = (10,10))

# Preprocessing

first we create dummy variables for categorial columns which  have more than two values, meaning `cp, thal and slope` and drop the original columns.

In [ ]:
a = pd.get_dummies(df['cp'], prefix = "cp")
b = pd.get_dummies(df['thal'], prefix = "thal")
c = pd.get_dummies(df['slope'], prefix = "slope")


frames = [df, a, b, c]
df = pd.concat(frames, axis = 1)
df = df.drop(columns = ['cp', 'thal', 'slope'])
df.head()

In [ ]:
df.columns

In [ ]:
# transform data to numeric to enable further analysis.
df = df.apply(pd.to_numeric)
df.dtypes

In [ ]:
x = df.drop(['target'], axis = 1)
y = df.target.values

In [ ]:
from sklearn.model_selection import train_test_split
# split data to train and test 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,stratify=y, random_state = 42)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
# convert the data to categorical labels

from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, num_classes=None)
y_test = to_categorical(y_test, num_classes=None)
print (y_train.shape)

In [ ]:
x_train.shape

# Neural Network 
## initialization

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense
import warnings

In [ ]:
classifier = Sequential()
# add input layer and hidden layer
classifier.add(Dense(
    activation="relu", input_dim=21, units=6,
    kernel_initializer="normal",kernel_regularizer=regularizers.l2(0.001)
))
classifier.add(Dropout(0.2))

# Adding the output layer
classifier.add(Dense(activation="softmax", units=2))
classifier.save_weights("weights")

## SGD Optimizer

In [ ]:
history = {}

In [ ]:
from keras.optimizers import SGD
classifier.load_weights("weights")
opt1 = SGD(lr=0.001)
classifier.compile(optimizer = opt1, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history['SGD'] = classifier.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 16, epochs = 50)

In [ ]:
# Predicting the Test set results
sgd_y_pred = classifier.predict(x_test)

In [ ]:
sgd_cr = classification_report(y_test,sgd_y_pred.round())
print(sgd_cr)

## Momentum SGD Optimizer

In [ ]:
# Compiling the ANN
from keras.optimizers import SGD
classifier.load_weights("weights")
opt2 = SGD(lr=0.001,momentum=0.9)
classifier.compile(optimizer = opt2, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history['momentum_SGD'] = classifier.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 10, epochs = 50)

In [ ]:
# Predicting the Test set results
msgd_y_pred = classifier.predict(x_test)

In [ ]:
msgd_cr = classification_report(y_test,msgd_y_pred.round())
print(msgd_cr)

## ADAM Optimizer

In [ ]:
from keras.optimizers import Adam
classifier.load_weights("weights")
opt3 = Adam(lr=0.001)
classifier.compile(optimizer = opt3, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history['Adam'] = classifier.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 10, epochs = 50)

In [ ]:
# Predicting the Test set results
adam_y_pred = classifier.predict(x_test)

In [ ]:
adam_cr = classification_report(y_test,adam_y_pred.round())
print(adam_cr)

## RMSprop Optimizer

In [ ]:
from keras.optimizers import RMSprop

classifier.load_weights("weights")
opt4 = RMSprop(lr=0.001,rho=0.9)
classifier.compile(optimizer = opt4 , loss = 'categorical_crossentropy', metrics = ['accuracy'])
history['RMSprop'] = classifier.fit(x_train, y_train,validation_data=(x_test, y_test), batch_size = 10, epochs = 50)

In [ ]:
rms_y_pred = classifier.predict(x_test)

In [ ]:
rms_cr = classification_report(y_test,rms_y_pred.round())
print(rms_cr)

In [ ]:
plt.subplots(figsize=(13, 8))
# plt.plot(history['RMSprop'].history['accuracy'])
plt.plot(history['SGD'].history['loss'],label='SGD')
plt.plot(history['momentum_SGD'].history['loss'],label='momentum_SGD')
plt.plot(history['Adam'].history['loss'],label = 'Adam')
plt.plot(history['RMSprop'].history['loss'],label = 'RMSprop')
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.subplots(figsize=(13, 8))
# plt.plot(history['RMSprop'].history['accuracy'])
plt.plot(history['SGD'].history['val_loss'],label='SGD')
plt.plot(history['momentum_SGD'].history['val_loss'],label='momentum_SGD')
plt.plot(history['Adam'].history['val_loss'],label = 'Adam')
plt.plot(history['RMSprop'].history['val_loss'],label = 'RMSprop')
plt.title('Model loss')
plt.ylabel('Validation loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()